In [1]:
import re
import sys
import pickle
from tqdm import tqdm

def make_split(line):
    if re.match(r'.*([，…?!\.,!？])$', ''.join(line)):#如果遇到这些字符，都返回空的
                                                        #join将序列中的元素以指定的字符连接生成一个新的字符串。
                                                        #str = "-";
                                                        #seq = ("a", "b", "c"); # 字符串序列
                                                        #print str.join( seq ) -> a-b-c
        #如：你好？我是xxx -> 你好我是xxx，
        return  []
    return [', ']

def good_line(line):#句子是否有用
    if len(re.findall(r'[a-zA-Z0-9]', ''.join(line)))>2:#0-9 a-z不能超过2个
        return False
    return True

def regular(sen):
    sen = re.sub(r'\.{3,100}', '…', sen)#sub替换，如果遇到3-100个.，都替换成...，sen是要处理的句子，
    sen = re.sub(r'…{2,100}', '…', sen)
    sen = re.sub(r'[,]{1,100}', '，', sen)
    sen = re.sub(r'[\.]{1,100}', '。', sen)
    sen = re.sub(r'[\?]{1,100}', '？', sen)
    sen = re.sub(r'[!]{1,100}', '！', sen)
    return sen

def main(limit=20, x_limit=3, y_limit=6):#一个句子最多20字符，
    from word_sequence import WordSequence

    print('extract lines')
    fp = open("dgk_shooter_min.conv", 'r', errors='ignore', encoding='utf-8')#读取文件
    groups = []
    group = []

    for line in tqdm(fp):#以进度条的方式读
        if line.startswith('M '):#M开头是句子，E开头是段落分隔
            line = line.replace('\n', '')#去掉回车

            if '/' in line:
                line = line[2:].split('/')
            else:
                line = list(line[2:])
            line = line[:-1]

            group.append(list(regular(''.join(line))))
        else:
            if group:
                groups.append(group)
                group = []#把当前段落加入进gruops，再让当前段落为空
    if group:#最后一段
        groups.append(group)
        group = []
    print('extract group')

    x_data = []
    y_data = []
    for group in tqdm(groups):
        for i, line in enumerate(group):
            last_line = None
            if i > 0:
                last_line = group[i - 1]
                if not good_line(last_line):
                    last_line = None
            next_line = None
            if i < len(group) - 1:
                next_line = group[i + 1]
                if not good_line(next_line):
                    next_line = None
            next_next_line = None
            if i < len(group) -2:
                next_next_line = group[i + 2]
                if not good_line(next_next_line):
                    next_next_line = None

            if next_line:
                x_data.append(line)
                y_data.append(next_line)
            if last_line and next_line:
                x_data.append(last_line + make_split(last_line) + line)
                y_data.append(next_line)
            if next_line and next_next_line:
                x_data.append(line)
                y_data.append(next_line + make_split(next_line) + next_next_line)

    print(len(x_data), len(y_data))

    for ask, answer in zip(x_data[:20], y_data[:20]):
        print(''.join(ask))
        print(''.join(answer))
        print('-'*20)

    data = list(zip(x_data, y_data))
    data = [
        (x, y)
        for x, y in data
        if len(x) < limit \
        and len(y) < limit \
        and len(y) >= y_limit \
        and len(x) >= x_limit
    ]
    x_data, y_data = zip(*data)

    print('fit word_sequence')

    ws_input = WordSequence()
    ws_input.fit(x_data + y_data)

    print('dump')

    pickle.dump((x_data, y_data),open('chatbot.pkl', 'wb'))#保存问答对训练数据
    pickle.dump(ws_input, open('ws.pkl', 'wb'))#保存词典模型

    print('done')

if __name__ == '__main__':
    main()

extract lines


4268087it [00:46, 91251.72it/s] 


extract group


100%|██████████| 762720/762720 [00:39<00:00, 19469.44it/s]


6196405 6196405
畹华吾侄
你接到这封信的时候
--------------------
畹华吾侄
你接到这封信的时候, 不知道大伯还在不在人世了
--------------------
你接到这封信的时候
不知道大伯还在不在人世了
--------------------
畹华吾侄, 你接到这封信的时候
不知道大伯还在不在人世了
--------------------
咱们梅家从你爷爷起
就一直小心翼翼地唱戏
--------------------
咱们梅家从你爷爷起
就一直小心翼翼地唱戏, 侍奉宫廷侍奉百姓
--------------------
就一直小心翼翼地唱戏
侍奉宫廷侍奉百姓
--------------------
咱们梅家从你爷爷起, 就一直小心翼翼地唱戏
侍奉宫廷侍奉百姓
--------------------
就一直小心翼翼地唱戏
侍奉宫廷侍奉百姓, 从来不曾遭此大祸
--------------------
侍奉宫廷侍奉百姓
从来不曾遭此大祸
--------------------
就一直小心翼翼地唱戏, 侍奉宫廷侍奉百姓
从来不曾遭此大祸
--------------------
侍奉宫廷侍奉百姓
从来不曾遭此大祸, 太后的万寿节谁敢不穿红
--------------------
从来不曾遭此大祸
太后的万寿节谁敢不穿红
--------------------
侍奉宫廷侍奉百姓, 从来不曾遭此大祸
太后的万寿节谁敢不穿红
--------------------
从来不曾遭此大祸
太后的万寿节谁敢不穿红, 就你胆儿大
--------------------
太后的万寿节谁敢不穿红
就你胆儿大
--------------------
从来不曾遭此大祸, 太后的万寿节谁敢不穿红
就你胆儿大
--------------------
太后的万寿节谁敢不穿红
就你胆儿大, 唉这我舅母出殡
--------------------
就你胆儿大
唉这我舅母出殡
--------------------
太后的万寿节谁敢不穿红, 就你胆儿大
唉这我舅母出殡
--------------------
fit word_sequence
dump
done
